In [2]:
# read shakespeare
with open('./data/input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print('char length of dataset: ', len(text))

# Sample text
print(text[:1000])

char length of dataset:  1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for br

In [3]:
# build vocabulary
chars = sorted(list(set(text)))
vocab_size = len(chars)
print('vocab: ' + ''.join(chars))
print(vocab_size)

vocab: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [4]:
# build encoder/decoder
# Google uses (kinda) SentencePiece, OpenAI uses (kinda) tiktoken
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output list of integers
decode = lambda l: ''.join(itos[i] for i in l) # decoder: take a list of integers, output a string

print(encode('hi there!'))
print(decode(encode('hi there!')))

# import tiktoken
# enc = tiktoken.get_encoding('gpt2')
# print('gpt2 vocab size:', enc.n_vocab)

[46, 47, 1, 58, 46, 43, 56, 43, 2]
hi there!


In [5]:


# encode the ENTIRE text dataset into a torch.Tensor
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype) # torch.Size([1115394]), torch.int64
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [6]:
# split data to train/val
n = int(0.9*len(data)) # first 90% will train
train_data = data[:n]
val_data = data[n:]

# also called context length, chunk size, etc
block_size = 8
print(train_data[:block_size+1])

# chunks...overlap...kind of?
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'when input is {context} the target: {target}')



tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])
when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [7]:


# batch (becuase chunks are small and we can process so many)
torch.manual_seed(1337)
batch_size = 4 # how many parallel processes?
block_size = 8 # what is the maximum context length?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape) # torch.Size([4, 8])
print(xb)
print('targets:')
print(yb.shape) # torch.Size([4, 8])
print(yb)
print('*'*50)

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f'when input is {context.tolist()} the target: {target}')


inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
**************************************************
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 5

In [8]:


# simplest neural network
# bigram langugage model
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off logits for the next token from lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C), or (Batch, Time (Block), Channel (Vocab_Size))

        if targets is None:
            loss = None
        else:
            # negative log loss
            # loss = F.cross_entropy(logits, targets) # https://docs.pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html
            # BASICALLY...cross entropy needs channels in second dim, let's reshape
            
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T) # or, .view(-1)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B,T) array of indices in current context
        for _ in range(max_new_tokens):
            # get predictions
            logits, loss = self(idx)

            # focus only on last time step
            logits = logits[:,-1,:] # becomes (B,C)

            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B,C)

            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B,1)

            # concat sampled index to running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B,T+1)

        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape) # torch.Size([4, 8, 65]) ORIGNALLY, then torch.Size([32, 65]) when reshaped
print(loss) # -ln(1/65) = 4.8786




torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)


In [9]:
# a zero object, just new line
idx = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx,max_new_tokens=100)[0].tolist()))



SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [10]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

batch_size = 32
for steps in range(10000):

    # sample a batch of data
    xb, yb = get_batch('train')
    
    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print('Loss: ',loss.item())
idx = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx,max_new_tokens=100)[0].tolist()))

Loss:  2.382369041442871

lso br. ave aviasurf my, yxMPZI ivee iuedrd whar ksth y h bora s be hese, woweee; the! KI 'de, ulsee


# the math trick in self-attention

In [11]:
# consider the following toy example:

B,T,C = 4,8,2 # batch, time (context), channels (vocab size)
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [12]:
# we want to couple the T tokens together, right now, they are seperate
# specifically, x[b,t] = mean_{i<=t} x[b,i]
# bow = bag-of-words
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)

In [14]:
x[0]

tensor([[ 0.6991,  1.9144],
        [-0.7969, -0.3562],
        [ 0.4291, -1.0265],
        [-0.8155, -0.8352],
        [ 0.8331, -0.0127],
        [ 0.8081, -0.9217],
        [ 0.3804,  0.6190],
        [-0.1611, -0.1540]])

In [15]:
xbow[0]

tensor([[ 0.6991,  1.9144],
        [-0.0489,  0.7791],
        [ 0.1104,  0.1773],
        [-0.1210, -0.0759],
        [ 0.0698, -0.0632],
        [ 0.1928, -0.2063],
        [ 0.2196, -0.0884],
        [ 0.1720, -0.0966]])